# Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack, load_npz, save_npz
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

import transformers as ppb # pytorch-pretrained-bert
import torch

import pipeline as pp
import models as ml

import importlib
import math
import pickle

: 

# Preproccessing

Covert types to binary labels - either True (reliable) or False (fake news)

In [ ]:
pp.apply_pipeline(
    "../datasets/big/dataset.csv", 
    [(pp.Binary_labels(), 'type', 'type_binary')], 
    new_file="../datasets/big/dataset_bin.csv", 
    progress_bar=True
)

: 

Create the follwoing input files:
* All are unbalanced
* The test and validation set are balanced according to the types (e.g. satire, reliable...), and the test set is unbalanced
* The test and validation set are balanced according to the binary classes, and the test set is unbalanced

In [6]:
# The number of rows to train the model
BATCH_SIZE = 5000

In [3]:
data = pd.read_csv("../datasets/big/news_sample_cleaned_num_100k.csv")

C:\Users\madsv\AppData\Local\Temp\ipykernel_20684\331561095.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../datasets/big/news_sample_cleaned_num_100k.csv")


In [4]:
data

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,6301449,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"['editor', 'drop', 'colleg', 'editori', 'march...",2018-02-11 00:40:10.316783,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,Time to Scrap the Electoral College?,NaN,NaN,"['Presidential Elections (US)', 'Electoral Col...",<br>To the Editor:\n<p>\n Re ''Drop Out of th...,NaN,NaN,nytimes
1,3676064,nationalreview.com,political,http://www.nationalreview.com/postmodern-conse...,"['ive', 'written', 'anoth', 'channel', 'advic'...",2017-11-27T01:14:42.983556,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,"Antonin Scalia, Donald Trump, Tyler Cowen & Co...",Peter Augustine Lawler,NaN,['Peter Augustine Lawler'],Senate Republicans should enter into the battl...,NaN,NaN,NaN
2,5192554,infowars.com,conspiracy,https://www.infowars.com/soldiers-nearly-kille...,"['david', 'gutierrez', 'natur', 'news', 'novem...",2017-12-09T22:10:08.302997,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Soldiers Nearly Killed with Military’s Bioterr...,NaN,NaN,[''],NaN,NaN,NaN,NaN
3,1136409,redstate.com,political,https://www.redstate.com/diary/Erick/2010/10/2...,"['colorado', 'close', 'call', 'right', 'union'...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Colorado Should Not Be Too Close To Call,"Erick Erickson, Redstate Insider, Susan Wright...",NaN,[''],Colorado Should Not Be Too Close To Call,"Michael Bennet, Colorado, ken buck",NaN,NaN
4,62174,beforeitsnews.com,fake,http://beforeitsnews.com/survival/2015/06/lett...,"['letter', 'number', 'beast', 'area', 'code', ...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Letter Re: Does the Number of the Beast Have a...,NaN,NaN,[''],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,7240944,nytimes.com,reliable,https://www.nytimes.com/2017/06/23/realestate/...,"['ye', 'ye', 'four', 'year', 'ago', 'children'...",2018-02-11 00:49:30.951722,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,A Brooklyn-Born Actress Rediscovers New York,"Joanne Kaufman, What I Love",NaN,"['Real Estate and Housing (Residential)', 'Int...",After giving up Manhattan to raise a family in...,NaN,NaN,nytimes
99999,2741248,lifenews.com,bias,https://consciouslifenews.com/category/conscio...,"['peopl', 'learn', 'rope', 'law', 'attract', '...",2017-11-27T01:14:21.395055,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Happiness & Humor Archives,"Tamara Rant, Andrea Schulman, Vicki Howie",NaN,[''],NaN,NaN,NaN,NaN
100000,6540290,nytimes.com,reliable,https://www.nytimes.com/2008/05/20/business/20...,"['low', 'compani', 'report', '<num>', 'percent...",2018-02-11 00:42:31.282156,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"Lowe’s, Hurt by the Slump in Housing and the E...",NaN,NaN,"[""Lowe's Companies"", 'Company Reports', 'Home ...","Lowe’s, the second-largest home improvement ch...",NaN,NaN,nytimes
100001,1405351,abovetopsecret.com,conspiracy,http://www.abovetopsecret.com/forum/thread5780...,"['investig', 'intens', 'infrar', 'emiss', 'tar...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"Oakland Ca. 05-28-10, page 4",NaN,NaN,[''],NaN,NaN,NaN,NaN


In [7]:
importlib.reload(pp)
from_file = "../datasets/big/shuffled.csv"

#pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [False, False, False], 
#                                   out_file="../datasets/sample/dataset_unbalanced.csv", get_frame=False)

pp.get_dataframe_with_distribution("../datasets/big/news_sample_cleaned_num_100k.csv", BATCH_SIZE, [0.8,0.1,0.1], [False, False, False], 
                                   out_file="../datasets/sample/dataset_unbalanced_new.csv", get_frame=False)



#pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.6,0.1,0.1], [True, False, False], 
#                                   out_file="../datasets/sample/dataset_balanced_types.csv", get_frame=False)
#pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [True, False, False],
#                                   out_file="../datasets/sample/dataset_balanced_bin.csv", get_frame=False, classes=[True,False], type_col="type_binary")
#pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [True, False, False], 
#                                   out_file="../datasets/sample/dataset_reliable_fake.csv", get_frame=False, classes=["reliable", "fake"])

c:\Users\madsv\Documents\Documents\University\DataScience\FakeNews\src\pipeline.py:145: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
100%|██████████| 100003/100003 [00:00<00:00, 799499.43it/s]


entries read: 200000


In [8]:
l = pd.read_csv("../datasets/sample/dataset_unbalanced_new.csv")

In [11]:
l

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,set
0,6301449,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"['editor', 'drop', 'colleg', 'editori', 'march...",2018-02-11 00:40:10.316783,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,Time to Scrap the Electoral College?,NaN,NaN,"['Presidential Elections (US)', 'Electoral Col...",<br>To the Editor:\n<p>\n Re ''Drop Out of th...,NaN,NaN,nytimes,0
1,3676064,nationalreview.com,political,http://www.nationalreview.com/postmodern-conse...,"['ive', 'written', 'anoth', 'channel', 'advic'...",2017-11-27T01:14:42.983556,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,"Antonin Scalia, Donald Trump, Tyler Cowen & Co...",Peter Augustine Lawler,NaN,['Peter Augustine Lawler'],Senate Republicans should enter into the battl...,NaN,NaN,NaN,0
2,5192554,infowars.com,conspiracy,https://www.infowars.com/soldiers-nearly-kille...,"['david', 'gutierrez', 'natur', 'news', 'novem...",2017-12-09T22:10:08.302997,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Soldiers Nearly Killed with Military’s Bioterr...,NaN,NaN,[''],NaN,NaN,NaN,NaN,0
3,1136409,redstate.com,political,https://www.redstate.com/diary/Erick/2010/10/2...,"['colorado', 'close', 'call', 'right', 'union'...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Colorado Should Not Be Too Close To Call,"Erick Erickson, Redstate Insider, Susan Wright...",NaN,[''],Colorado Should Not Be Too Close To Call,"Michael Bennet, Colorado, ken buck",NaN,NaN,0
4,62174,beforeitsnews.com,fake,http://beforeitsnews.com/survival/2015/06/lett...,"['letter', 'number', 'beast', 'area', 'code', ...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Letter Re: Does the Number of the Beast Have a...,NaN,NaN,[''],NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4692181,beforeitsnews.com,fake,http://beforeitsnews.com/spirit/2016/11/thane-...,"['two', 'remot', 'villag', 'neighbour', 'thane...",2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Thane villages get rid of darkness this Diwali,Wisdom Blog Of Art Of Living,NaN,[''],NaN,NaN,NaN,NaN,2
4996,1308332,chroniclesmagazine.org,political,http://www.chroniclesmagazine.org/1998/Februar...,"['peter', 'lababera', 'publish', 'lambda', 're...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Chronicles Magazine,NaN,NaN,[''],NaN,NaN,NaN,NaN,2
4997,6505182,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"['saintsa', '?', 'piano', 'concerto', 'no', '<...",2018-02-11 00:42:12.085863,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"Classical Recordings: Ambient Haze, Romanticis...",NaN,NaN,[''],SAINT-SA�NS: PIANO CONCERTOS NOS. 2 & 5; FRANC...,NaN,NaN,nytimes,2
4998,73138,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,"['barack', 'terribl', 'horribl', 'good', 'bad'...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Barack and the Terrible, Horrible, No Good, Ve...",NaN,NaN,[''],NaN,NaN,NaN,NaN,2


Check distribution of labels (just to show that everything works)

In [135]:
def get_distribution(data, is_percentage=True, col = "type"):
    for i, label in enumerate(pp.labels):
        if is_percentage:
            percent = len(data[data[col] == label]) / (data.shape[0])
        else:
            percent = len(data[data[col] == label])
        print(f"{label}: {percent}", end="")
        print(", ", end="") if i != len(pp.labels) - 1 else _

for file in ["../datasets/big/dataset_unbalanced.csv", "../datasets/big/dataset_balanced_types.csv", "../datasets/big/dataset_balanced_bin.csv"]:
    data = pd.read_csv(file)
    print(f"File: {file} ----------------------------------")
    # find distribution of labels
    for i, set_name in enumerate(["train", "val", "test"]):
        set = data[data["set"] == i]
        print(f"Distribution of {set_name} with size {set.shape[0]}:")
        get_distribution(set)
        print(f"\nTrue: {len(set[set['type_binary'] == True])}, Fake: {len(set[set['type_binary'] == False])}")

File: ../datasets/big/dataset_unbalanced.csv ----------------------------------
Distribution of train with size 8000:
fake: 0.121625, conspiracy: 0.123125, junksci: 0.0155, hate: 0.01, unreliable: 0.04575, bias: 0.15475, satire: 0.016125, reliable: 0.259125, clickbait: 0.032, political: 0.222
True: 4105, Fake: 3895
Distribution of val with size 1000:
fake: 0.129, conspiracy: 0.124, junksci: 0.012, hate: 0.014, unreliable: 0.045, bias: 0.147, satire: 0.022, reliable: 0.262, clickbait: 0.021, political: 0.224
True: 507, Fake: 493
Distribution of test with size 1000:
fake: 0.106, conspiracy: 0.125, junksci: 0.017, hate: 0.008, unreliable: 0.05, bias: 0.172, satire: 0.014, reliable: 0.255, clickbait: 0.032, political: 0.221
True: 508, Fake: 492
File: ../datasets/big/dataset_balanced_types.csv ----------------------------------
Distribution of train with size 8000:
fake: 0.1, conspiracy: 0.1, junksci: 0.1, hate: 0.1, unreliable: 0.1, bias: 0.1, satire: 0.1, reliable: 0.1, clickbait: 0.1, po

Cleaning the files

In [14]:
importlib.reload(pp)

def Clean_data(file, new_file):
    stopwords_lst = stopwords.words('english')
    pp.apply_pipeline(file, [
            # binary labels
            (pp.Binary_labels(), 'type', 'type_binary'),
            # Clean content
            (pp.Clean_data(), 'content'),
            (pp.Tokenizer(), "content"),
            (pp.Remove_stopwords(stopwords_lst), "content"),
            (pp.Stem(), "content"),
            (pp.Combine_Content(), "content", "content_combined"),
            # Clean authors
            (pp.Clean_author(), "authors"),
            # Clean title
            (pp.Clean_data(), 'title'),
            (pp.Tokenizer(), "title"),
            (pp.Remove_stopwords(stopwords_lst), "title"),
            (pp.Stem(), "title"),
            (pp.Combine_Content(), "title"),
            # Clean domain
            (pp.Clean_domain(), 'domain'),
            # Combine columns (used as features)
            (pp.Join_str_columns(["content_combined", "authors"]), None, "content_authors"),
            (pp.Join_str_columns(["content_combined", "title"]), None, "content_title"),
            (pp.Join_str_columns(["content_combined", "domain"]), None, "content_domain"),
            (pp.Join_str_columns(["content_combined", "domain", "authors", "title"]), None, "content_domain_authors_title")
        ],
        new_file=new_file,
        progress_bar=True,
    )

Clean_data("../datasets/sample/dataset_unbalanced.csv", "../datasets/sample/dataset_unbalanced_cleaned.csv")
#Clean_data("../datasets/sample/dataset_balanced_types.csv", "../datasets/sample/dataset_balanced_types_cleaned.csv")
#Clean_data("../datasets/sample/dataset_balanced_bin.csv", "../datasets/sample/dataset_balanced_bin_cleaned.csv")
#Clean_data("../datasets/sample/dataset_balanced_bin.csv", "../datasets/sample/dataset_balanced_bin_cleaned.csv")

100%|██████████| 5000/5000 [00:00<00:00, 29124.24it/s]


processed 200000 rows
finish time: 50.075483083724976


In [107]:
def split_data(data, features, y, set="set", get_val=True):
    train = data[data[set] == 0]
    val = data[data[set] == 1]
    test = data[data[set] == 2]
    X_train, y_train = train[features], (train[y].astype(int))
    X_val, y_val = val[features], (val[y].astype(int))
    X_test, y_test = test[features], (test[y].astype(int))
    if not get_val:
        return X_train, X_test, y_train, y_test
    return X_train, X_val, X_test, y_train, y_val, y_test

In [108]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(pd.read_csv("../datasets/sample/dataset_unbalanced_cleaned.csv"), "content_combined", "type_binary")

# Training the logistic model

In [4]:
def create_count_vector(X_train, X_val, X_test):
    # count vectri
    count_vectorizer = CountVectorizer(ngram_range=(1, 1)) # unigram

    # fit and transform train data to count vectorizer
    count_vectorizer.fit(X_train)

    return (count_vectorizer.transform(X_train),
            count_vectorizer.transform(X_val),
            count_vectorizer.transform(X_test))

In [5]:
def create_tdfidf_vector(X_train, X_val, X_test, ngram_range=(1, 1)):
    # tfidf vector
    tfidf_vectorizer = TfidfVectorizer(ngram_range=ngram_range) # unigram

    # fit and transform train data to tfidf vectorizer
    tfidf_vectorizer.fit(X_train)

    return (tfidf_vectorizer.transform(X_train),
            tfidf_vectorizer.transform(X_val),
            tfidf_vectorizer.transform(X_test))

In [71]:
X_train_vec, X_val_vec, X_test_vec = create_tdfidf_vector(X_train, X_val, X_test)

In [96]:
def save_csr_picle(file, vectors, append=False):
    file_param = "wb" if not append else "ab"
    with open(file, file_param) as f:
        for train, val, test in vectors:
            pickle.dump(train, f)
            pickle.dump(val, f)
            pickle.dump(test, f)

def load_csr_picle(file, vector_names):
    vectors = {}
    with open(file, 'rb') as f:
        for names in vector_names:
            vectors[names] = (pickle.load(f), pickle.load(f), pickle.load(f))
        
    return vectors

In [101]:
# Vectorize data
def create_vector_file(file, vec_funcs, X_train, X_val, X_test, y_train, y_val, y_test):
    # save y
    save_csr_picle(file, [(y_train, y_val, y_test)])
    # vectorize X and save
    for func in vec_funcs:
        X_train_vec, X_val_vec, X_test_vec = func(X_train, X_val, X_test)
        save_csr_picle(file, [(X_train_vec, X_val_vec, X_test_vec)], append=True)
        print(f"Saved {func.__name__} vectors")


In [102]:
vec_funcs = [
    create_count_vector,
    lambda a, b, c: create_tdfidf_vector(a, b, c, ngram_range=(1, 1)),
    lambda a, b, c: create_tdfidf_vector(a, b, c, ngram_range=(1, 2)),
]
create_vector_file("../datasets/sample/dataset_unbalanced_cleaned_vectors.pickle", vec_funcs, X_train, X_val, X_test, y_train, y_val, y_test)

Saved create_count_vector vectors
Saved <lambda> vectors
Saved <lambda> vectors


In [109]:
vectors = load_csr_picle("../datasets/sample/dataset_unbalanced_cleaned_vectors.pickle", ["y", "count", "tfidf", "tfidf_bigram"])
X_train_1, X_val_1, X_test_1 = vectors["count"]
y_train_1, y_val_1, y_test_1 = vectors["y"]
try_models([LogisticRegression()], X_test_1, X_val_1, y_test, y_val)

LogisticRegression finished in 0.74 seconds


,name,train_acc,val_acc,precision,recall,f1,time
0,LogisticRegression,1.0,0.69,0.793103,0.587591,0.675052,0.74


In [103]:
def try_models(models, X_train, X_val, y_train, y_val, name=None):
    metrics = []
    for model in models:
        start_time = time() 
        model.fit(X_train, y_train)
        train_time = time() - start_time
        y_train_pred = model.predict(X_train)
        y_pred = model.predict(X_val)
        
        if name == None:
            name = type(model).__name__
        metrics.append({
            "name": name,
            "train_acc": accuracy_score(y_train, y_train_pred),
            "val_acc": accuracy_score(y_val, y_pred),
            "precision": precision_score(y_val, y_pred),
            "recall": recall_score(y_val, y_pred),
            "f1": f1_score(y_val, y_pred), 
            "time": "{:.2f}".format(train_time)
        })
        print(f"{name} finished in {(time() - start_time):.2f} seconds")
    return pd.DataFrame(metrics)

In [81]:
class Test_statistic():
    def __init__(self):
        self.metrics = pd.DataFrame()

    def test_baseline(self, X_train, X_val, y_train, y_val, name=None, model=None):
        if model == None:
            model = LogisticRegression()
        metric = try_models([model], X_train, X_val, y_train, y_val, name=name)
        self.metrics = pd.concat([self.metrics, metric])

    def test_col(self, data, col, name, model=None):
        self.test_baseline(*split_csr_data(data, features=col, y="type_binary", get_val=False), name=name, model=model)

    def test_cols(self, data, cols_to_test, model=None):
        for col, name in cols_to_test:
            self.test_baseline(*split_csr_data(data, features=col, y="type_binary", get_val=False), name=name, model=model)

In [82]:
def test_files(files, cols_to_test, vec_funcs, tests = None):
    if tests == None:
        tests = Test_statistic()
    for file, name in files:
        print(f"Proccessing: {name}")
        cols_to_read = list(list(zip(*cols_to_test))[0]) + ["type_binary", "set"]
        data = pd.read_csv(file, usecols=cols_to_read)
        print("Read data into dataframe")

        for col, entry_name in cols_to_test:
            for func, model, func_name in vec_funcs:
                X_train, X_val, X_test, y_train, y_val, y_test = split_data(data, col, "type_binary")
                X_train_vec, X_val_vec, X_test_vec = func(X_train, X_val, X_test)
                print(f"Vectorized {entry_name} with {func_name}")
                #print(X_train_vec.shape, X_val_vec.shape, X_test_vec.shape, y_train.shape, y_val.shape, y_test.shape)
                tests.test_baseline(X_train_vec, X_val_vec, y_train, y_val, name=f"{entry_name}_{name}_{func_name}", model=model)
    return tests

In [49]:
test_stat = test_files(
    files = [
        ("../datasets/sample/dataset_unbalanced_cleaned.csv", ""), 
    ],
    cols_to_test = [
        ("content_combined", "content"),
        ("content_authors", "content_authors"), 
        ("content_title", "content_title"),
        ("content_domain", "content_domain"),
        ("content_domain_authors_title", "all")
    ],
    vec_funcs = [
        (create_count_vector, LogisticRegression(max_iter=1000), "count_vec"),
        (lambda a, b, c: create_tdfidf_vector(a, b, c, ngram_range=(1, 1)), LogisticRegression(), "tfidf_vec"),
        (lambda a, b, c: create_tdfidf_vector(a, b, c, ngram_range=(1, 2)), LogisticRegression(), "tfidf_vec_bigram"),
    ])
test_stat.metrics.sort_values(by="f1", ascending=False)

Proccessing: 
Read data into dataframe
Vectorized content with count_vec
content__count_vec finished in 1.76 seconds
Vectorized content with tfidf_vec
content__tfidf_vec finished in 0.39 seconds
Vectorized content with tfidf_vec_bigram
content__tfidf_vec_bigram finished in 5.58 seconds
Vectorized content_authors with count_vec
content_authors__count_vec finished in 1.70 seconds
Vectorized content_authors with tfidf_vec
content_authors__tfidf_vec finished in 0.40 seconds
Vectorized content_authors with tfidf_vec_bigram
content_authors__tfidf_vec_bigram finished in 4.99 seconds
Vectorized content_title with count_vec
content_title__count_vec finished in 1.61 seconds
Vectorized content_title with tfidf_vec
content_title__tfidf_vec finished in 0.32 seconds
Vectorized content_title with tfidf_vec_bigram
content_title__tfidf_vec_bigram finished in 6.40 seconds
Vectorized content_domain with count_vec
content_domain__count_vec finished in 1.64 seconds
Vectorized content_domain with tfidf_vec


,name,train_acc,val_acc,precision,recall,f1,time
0,content__count_vec,0.99950,0.764,0.814516,0.737226,0.773946,1.75
0,content__tfidf_vec,0.93475,0.784,0.809701,0.791971,0.800738,0.38
0,content__tfidf_vec_bigram,0.98050,0.800,0.815217,0.821168,0.818182,5.56
0,content_authors__count_vec,0.99975,0.798,0.841897,0.777372,0.808349,1.69
0,content_authors__tfidf_vec,0.94700,0.816,0.842105,0.817518,0.829630,0.39
0,content_authors__tfidf_vec_bigram,0.98350,0.822,0.833935,0.843066,0.838475,4.97
0,content_title__count_vec,1.00000,0.784,0.826772,0.766423,0.795455,1.61
0,content_title__tfidf_vec,0.93825,0.794,0.813187,0.810219,0.811700,0.31
0,content_title__tfidf_vec_bigram,0.98025,0.792,0.805755,0.817518,0.811594,6.38
0,content_domain__count_vec,1.00000,0.846,0.880309,0.832117,0.855535,1.63


Old:

In [48]:
# contains the files to test and the name of the test-group (for the dataframe)
files = [
    ("../datasets/sample/dataset_unbalanced_cleaned.csv", "unbalanced"), 
    #("../datasets/big/dataset_unbalanced_cleaned.csv", "unbalanced"), 
    #("../datasets/big/dataset_balanced_types_cleaned.csv", "balanced_types"), 
    #("../datasets/big/dataset_balanced_bin_cleaned.csv", "balanced_bin"),
    #("../datasets/big/dataset_reliable_fake_cleaned.csv", "reliable_fake")
],
# contains the columns to test and the name of the specific test (for the dataframe)
cols_to_test = [
    ("content_combined", "content"),
    ("content_authors", "content_authors"), 
    ("content_title", "content_title"),
    ("content_domain", "content_domain"),
    ("content_domain_authors_title", "all")
]

In [12]:
for file, name in files:
    print(f"Proccessing: {name}")
    cols_to_read = list(list(zip(*cols_to_test))[0]) + ["type_binary", "set"]
    vectorized_data = pd.read_csv(file, usecols=cols_to_read)
    print("Read data into dataframe")
    for col, entry_name in cols_to_test:
        #vectorize_content(vectorized_data, col=col, new_col=f"{col}_vectorized")
        #tests.test_col(vectorized_data, f"{col}_vectorized", f"{entry_name}_{name}")
        vectorize_content(vectorized_data, col=col, new_col=col)
        tests.test_col(vectorized_data, col, f"{entry_name}_{name}")
        del vectorized_data[col] # free up memory

Proccessing: unbalanced
Read data into dataframe


In [52]:
tests.metrics.sort_values(by="f1", ascending=False)

,name,train_acc,val_acc,precision,recall,f1,time
0,all_unbalanced_count_vec,1.00000,0.846,0.871698,0.843066,0.857143,1.86
0,content_domain_unbalanced_count_vec,1.00000,0.846,0.880309,0.832117,0.855535,1.49
0,content_authors_unbalanced_count_vec,0.99975,0.798,0.841897,0.777372,0.808349,1.97
0,content_title_unbalanced_count_vec,1.00000,0.784,0.826772,0.766423,0.795455,1.73
0,content_unbalanced_count_vec,0.99950,0.764,0.814516,0.737226,0.773946,1.79


Best file and features

In [ ]:
best_file = "../datasets/big/dataset_unbalanced_cleaned.csv"
best_col = "content_domain_authors_title"

In [271]:
data = pd.read_csv(best_file)
vectorize_content(data, col=best_col, new_col=best_col)

Time elapsed of TF IDF transform for content_domain_authors_title: 9.814547061920166


Hyperparameter tuning - the best found was C=300 and max_iter=200

In [ ]:
model = LogisticRegression()
param_grid = {"C": [250, 300, 350], "max_iter": [150, 200, 250]} #200 won - det samme
#param_grid = {'penalty': ['l1', 'l2'],'C': [350], "maxiter": [200], 'solver': ['liblinear', 'saga']}

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    cv=3,
                    scoring=['f1'],
                    refit='f1',
                    verbose=2) #'accuracy'


tests.test_col(data, best_col, "hyper_1", model=grid)
print(grid.best_estimator_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ................................C=300, max_iter=100; total time=  10.0s
[CV] END ................................C=300, max_iter=100; total time=  10.8s
[CV] END ................................C=300, max_iter=100; total time=   7.1s
[CV] END ................................C=300, max_iter=150; total time=   8.8s
[CV] END ................................C=300, max_iter=150; total time=  11.2s
[CV] END ................................C=300, max_iter=150; total time=   6.7s
[CV] END ................................C=300, max_iter=200; total time=   8.6s
[CV] END ................................C=300, max_iter=200; total time=  10.6s
[CV] END ................................C=300, max_iter=200; total time=   7.2s
hyper_1 finished in 88.89 seconds
LogisticRegression(C=300)


In [ ]:
data = pd.read_csv(best_file)
vectorize_content(data, col=best_col, new_col=best_col)
tests.test_col(data, best_col, "hyper_all", model=LogisticRegression(C=300, max_iter=200))
# test the best parameters on the other featues
vectorize_content(data, col="content_combined", new_col="content_combined")
tests.test_col(data, "content_combined", "hyper_content", model=LogisticRegression(C=300, max_iter=200))

hyper_all finished in 8.66 seconds
Time elapsed of TF IDF transform for content_combined: 7.822778701782227
hyper_content finished in 9.59 seconds


In [ ]:
tests.metrics.sort_values(by="f1", ascending=False)

,name,train_acc,test_acc,precision,recall,f1,time
0,content_domain_reliable_fake,0.9965,0.934,0.979332,0.920777,0.949153,4.29
0,content_domain_authors_title_reliable_fake,0.9970,0.929,0.985390,0.907324,0.944747,4.51
0,content_authors_reliable_fake,0.9955,0.903,0.976667,0.875934,0.923562,3.97
0,content_reliable_fake,0.9955,0.898,0.970149,0.874439,0.919811,3.21
0,content_title_reliable_fake,0.9955,0.896,0.976391,0.865471,0.917591,4.10
0,hyper_all,1.0000,0.867,0.881553,0.863118,0.872238,8.65
0,hyper_all,1.0000,0.867,0.881553,0.863118,0.872238,8.21
0,content_domain_authors_title_balanced_bin,0.9960,0.841,0.854127,0.842803,0.848427,4.34
0,content_domain_authors_title_unbalanced,0.9960,0.838,0.838290,0.857414,0.847744,5.14
0,content_domain_balanced_bin,0.9950,0.830,0.836466,0.842803,0.839623,4.29
